In [1]:
%config Completer.use_jedi = False
import pandas as pd
import numpy as np
pd.options.display.max_rows=1000
pd.options.display.max_columns=1000
import math
import re
import datetime as dt
from bureau_fc import get_bureau_feats
from multiprocessing import Pool
import warnings
warnings.filterwarnings("ignore")
import copy
from sklearn.metrics import f1_score

In [2]:
import sys
sys.path.append('ml_lib/')
from encoding import FreqeuncyEncoding
from custom_classifier_mutliclass import Estimator
from hyperopt_multiclass import HyperOptModelSelection
from hyperopt import hp
from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier

In [3]:
DATA_DIR = '/home/sahil/data/av/AV_LTFS3/data/'

In [4]:
train = pd.read_csv(DATA_DIR+'train_Data.csv')
test = pd.read_csv(DATA_DIR+'test_Data.csv')
bur_df = pd.read_pickle('bureau_future_feats_start_2_maturity_plus_2.pkl')
bur_df2 = pd.read_pickle('ltfs_bureau_future_feats_start_2_maturity_plus_2.pkl')
bur_df3 = pd.read_pickle('bur_future_feats.pkl')
bur_df4 = pd.read_pickle('bureau_lead_lag_numeric_feats.pkl')

In [5]:
target_map = {'No Top-up Service': 0,
 '12-18 Months': 1,
 '18-24 Months': 2,
 '24-30 Months': 3,
 '30-36 Months': 4,
 '36-48 Months': 5,
 ' > 48 Months': 6,
 }
train['Top-up Month'] = train['Top-up Month'].map(target_map)

In [6]:
reverse_map = {v:k for k,v in target_map.items()}

In [7]:
reverse_map

{0: 'No Top-up Service',
 1: '12-18 Months',
 2: '18-24 Months',
 3: '24-30 Months',
 4: '30-36 Months',
 5: '36-48 Months',
 6: ' > 48 Months'}

In [8]:
train['Top-up Month'].value_counts()

0    106677
6      8366
5      3656
3      3492
4      3062
2      2368
1      1034
Name: Top-up Month, dtype: int64

In [9]:
df = pd.concat((train,test),axis=0)
df.shape

(143400, 26)

In [10]:
# bur_df.shape

In [11]:
df = df.merge(bur_df,on='ID',how='left').merge(bur_df2,on='ID',how='left').merge(bur_df3,on='ID',how='left').merge(bur_df4,on='ID',how='left')
df['DisbursalDate'] = pd.to_datetime(df['DisbursalDate'])
df['MaturityDAte'] = pd.to_datetime(df['MaturityDAte'])

In [12]:
df.shape

(143400, 1305)

In [13]:
df.head()

,ID,Frequency,InstlmentMode,LoanStatus,PaymentMode,BranchID,Area,Tenure,AssetCost,AmountFinance,DisbursalAmount,EMI,DisbursalDate,MaturityDAte,AuthDate,AssetID,ManufacturerID,SupplierID,LTV,SEX,AGE,MonthlyIncome,City,State,ZiPCODE,Top-up Month,individual_accounts,joint_accounts,guarantor_accounts,curr_bal_grtr_0,num_accounts,mean_correctedDISBURSED-AMT/HIGH CREDIT,min_correctedDISBURSED-AMT/HIGH CREDIT,max_correctedDISBURSED-AMT/HIGH CREDIT,std_correctedDISBURSED-AMT/HIGH CREDIT,sum_correctedDISBURSED-AMT/HIGH CREDIT,mean_correctedCURRENT-BAL,min_correctedCURRENT-BAL,max_correctedCURRENT-BAL,std_correctedCURRENT-BAL,sum_correctedCURRENT-BAL,mean_correctedOVERDUE-AMT,min_correctedOVERDUE-AMT,max_correctedOVERDUE-AMT,std_correctedOVERDUE-AMT,sum_correctedOVERDUE-AMT,num_closed_accounts,num_open_accounts,num_delinq_accounts,total_written_off_amount,mean_percent_paid_off,min_percent_paid_off,max_percent_paid_off,std_percent_paid_off,overall_percent_paid_off,median_tenor,max_tenor,min_tenor,std_count,ddd_count,xxx_count,late_count,_30_count,_60_count,_90_count,_180_count,total_count,std_count_application_loan,ddd_count_application_loan,xxx_count_application_loan,late_count_application_loan,_30_count_application_loan,_60_count_application_loan,_90_count_application_loan,_180_count_application_loan,total_count_application_loan,mean_dpd_str,min_dpd_str,max_dpd_str,std_dpd_str,mean_dpd_str_application_loan,min_dpd_str_application_loan,max_dpd_str_application_loan,std_dpd_str_application_loan,num_accounts__between_0_and_365_days,total_sanctioned_amount__between_0_and_365_days,total_curr_bal__between_0_and_365_days,overall_percentage_paid_off__between_0_and_365_days,num_accounts__between_365_and_730_days,total_sanctioned_amount__between_365_and_730_days,total_curr_bal__between_365_and_730_days,overall_percentage_paid_off__between_365_and_730_days,num_accounts__between_730_and_1095_days,total_sanctioned_amount__between_730_and_1095_days,total_curr_bal__between_730_and_1095_days,overall_percentage_paid_off__between_730_and_1095_days,num_accounts__between_1095_and_1460_days,total_sanctioned_amount__between_1095_and_1460_days,total_curr_bal__between_1095_and_1460_days,overall_percentage_paid_off__between_1095_and_1460_days,num_accounts__between_1460_and_3650_days,total_sanctioned_amount__between_1460_and_3650_days,total_curr_bal__between_1460_and_3650_days,overall_percentage_paid_off__between_1460_and_3650_days,num_accounts_Tractor Loan,total_sanctioned_amount_Tractor Loan,total_curr_bal_Tractor Loan,overall_percentage_paid_off_Tractor Loan,num_accounts_Gold Loan,total_sanctioned_amount_Gold Loan,total_curr_bal_Gold Loan,overall_percentage_paid_off_Gold Loan,num_accounts_Business Loan Priority Sector Agriculture,total_sanctioned_amount_Business Loan Priority Sector Agriculture,total_curr_bal_Business Loan Priority Sector Agriculture,overall_percentage_paid_off_Business Loan Priority Sector Agriculture,num_accounts_Kisan Credit Card,total_sanctioned_amount_Kisan Credit Card,total_curr_bal_Kisan Credit Card,overall_percentage_paid_off_Kisan Credit Card,num_accounts_Auto Loan (Personal),total_sanctioned_amount_Auto Loan (Personal),total_curr_bal_Auto Loan (Personal),overall_percentage_paid_off_Auto Loan (Personal),num_accounts_Personal Loan,total_sanctioned_amount_Personal Loan,total_curr_bal_Personal Loan,overall_percentage_paid_off_Personal Loan,num_accounts_Other,total_sanctioned_amount_Other,total_curr_bal_Other,overall_percentage_paid_off_Other,num_accounts_Overdraft,total_sanctioned_amount_Overdraft,total_curr_bal_Overdraft,overall_percentage_paid_off_Overdraft,mean_day_start_day_diff_app_vs_other,min_day_start_day_diff_app_vs_other,max_day_start_day_diff_app_vs_other,std_day_start_day_diff_app_vs_other,mean_days_bw_loans,min_days_bw_loans,max_days_bw_loans,std_days_bw_loans,sum_days_bw_loans,ltfs_individual_accounts,ltfs_joint_accounts,ltfs_guarantor_accounts,ltfs_curr_bal_grtr_0,ltfs_num_accounts,ltfs_mean_correctedDISBURSED-AMT/HIGH CR

In [14]:
df['pin1'] = df['ZiPCODE'].fillna('000000').astype('str').str[:2]
df['pin2'] = df['ZiPCODE'].fillna('000000').astype('str').str[2:4]
df['pin3'] = df['ZiPCODE'].fillna('000000').astype('str').str[4:6:]
df['cal_tenor'] = (df['MaturityDAte'] - df['DisbursalDate']).dt.days
df['emi_sal_ratio'] = df['EMI']/df['MonthlyIncome']
df['ltv2'] = df['DisbursalAmount']/df['AssetCost']

In [15]:
df['disb_year'] = df['DisbursalDate'].dt.year
df['disb_mon'] = df['DisbursalDate'].dt.month
df['disb_day'] = df['DisbursalDate'].dt.day
df['disb_dow'] = df['DisbursalDate'].dt.dayofweek

df['mat_year'] = df['MaturityDAte'].dt.year
df['mat_mon'] = df['MaturityDAte'].dt.month
df['mat_day'] = df['MaturityDAte'].dt.day
df['mat_dow'] = df['MaturityDAte'].dt.dayofweek

In [16]:
cat_cols = ['Frequency', 'InstlmentMode', 'LoanStatus', 'PaymentMode', 'Area','ManufacturerID',
            'SupplierID','pin1','pin2','pin3', 'SEX', 'City', 'State','BranchID',
           ]
target = 'Top-up Month'
drop_cols = ['ID','DisbursalDate','MaturityDAte','AuthDate','AssetID','ZiPCODE']
num_cols = (df.columns[~df.columns.isin([target]+drop_cols+cat_cols)].tolist())
use_cols = cat_cols+num_cols

In [17]:
# df.columns[df.columns.str.contains('is_ltfs')]

In [18]:
len(use_cols)

1312

In [19]:
fe = FreqeuncyEncoding(categorical_columns=cat_cols,normalize=True,return_df=True)
df = fe.fit_transform(df)

In [20]:
df.head()

,ID,Frequency,InstlmentMode,LoanStatus,PaymentMode,BranchID,Area,Tenure,AssetCost,AmountFinance,DisbursalAmount,EMI,DisbursalDate,MaturityDAte,AuthDate,AssetID,ManufacturerID,SupplierID,LTV,SEX,AGE,MonthlyIncome,City,State,ZiPCODE,Top-up Month,individual_accounts,joint_accounts,guarantor_accounts,curr_bal_grtr_0,num_accounts,mean_correctedDISBURSED-AMT/HIGH CREDIT,min_correctedDISBURSED-AMT/HIGH CREDIT,max_correctedDISBURSED-AMT/HIGH CREDIT,std_correctedDISBURSED-AMT/HIGH CREDIT,sum_correctedDISBURSED-AMT/HIGH CREDIT,mean_correctedCURRENT-BAL,min_correctedCURRENT-BAL,max_correctedCURRENT-BAL,std_correctedCURRENT-BAL,sum_correctedCURRENT-BAL,mean_correctedOVERDUE-AMT,min_correctedOVERDUE-AMT,max_correctedOVERDUE-AMT,std_correctedOVERDUE-AMT,sum_correctedOVERDUE-AMT,num_closed_accounts,num_open_accounts,num_delinq_accounts,total_written_off_amount,mean_percent_paid_off,min_percent_paid_off,max_percent_paid_off,std_percent_paid_off,overall_percent_paid_off,median_tenor,max_tenor,min_tenor,std_count,ddd_count,xxx_count,late_count,_30_count,_60_count,_90_count,_180_count,total_count,std_count_application_loan,ddd_count_application_loan,xxx_count_application_loan,late_count_application_loan,_30_count_application_loan,_60_count_application_loan,_90_count_application_loan,_180_count_application_loan,total_count_application_loan,mean_dpd_str,min_dpd_str,max_dpd_str,std_dpd_str,mean_dpd_str_application_loan,min_dpd_str_application_loan,max_dpd_str_application_loan,std_dpd_str_application_loan,num_accounts__between_0_and_365_days,total_sanctioned_amount__between_0_and_365_days,total_curr_bal__between_0_and_365_days,overall_percentage_paid_off__between_0_and_365_days,num_accounts__between_365_and_730_days,total_sanctioned_amount__between_365_and_730_days,total_curr_bal__between_365_and_730_days,overall_percentage_paid_off__between_365_and_730_days,num_accounts__between_730_and_1095_days,total_sanctioned_amount__between_730_and_1095_days,total_curr_bal__between_730_and_1095_days,overall_percentage_paid_off__between_730_and_1095_days,num_accounts__between_1095_and_1460_days,total_sanctioned_amount__between_1095_and_1460_days,total_curr_bal__between_1095_and_1460_days,overall_percentage_paid_off__between_1095_and_1460_days,num_accounts__between_1460_and_3650_days,total_sanctioned_amount__between_1460_and_3650_days,total_curr_bal__between_1460_and_3650_days,overall_percentage_paid_off__between_1460_and_3650_days,num_accounts_Tractor Loan,total_sanctioned_amount_Tractor Loan,total_curr_bal_Tractor Loan,overall_percentage_paid_off_Tractor Loan,num_accounts_Gold Loan,total_sanctioned_amount_Gold Loan,total_curr_bal_Gold Loan,overall_percentage_paid_off_Gold Loan,num_accounts_Business Loan Priority Sector Agriculture,total_sanctioned_amount_Business Loan Priority Sector Agriculture,total_curr_bal_Business Loan Priority Sector Agriculture,overall_percentage_paid_off_Business Loan Priority Sector Agriculture,num_accounts_Kisan Credit Card,total_sanctioned_amount_Kisan Credit Card,total_curr_bal_Kisan Credit Card,overall_percentage_paid_off_Kisan Credit Card,num_accounts_Auto Loan (Personal),total_sanctioned_amount_Auto Loan (Personal),total_curr_bal_Auto Loan (Personal),overall_percentage_paid_off_Auto Loan (Personal),num_accounts_Personal Loan,total_sanctioned_amount_Personal Loan,total_curr_bal_Personal Loan,overall_percentage_paid_off_Personal Loan,num_accounts_Other,total_sanctioned_amount_Other,total_curr_bal_Other,overall_percentage_paid_off_Other,num_accounts_Overdraft,total_sanctioned_amount_Overdraft,total_curr_bal_Overdraft,overall_percentage_paid_off_Overdraft,mean_day_start_day_diff_app_vs_other,min_day_start_day_diff_app_vs_other,max_day_start_day_diff_app_vs_other,std_day_start_day_diff_app_vs_other,mean_days_bw_loans,min_days_bw_loans,max_days_bw_loans,std_days_bw_loans,sum_days_bw_loans,ltfs_individual_accounts,ltfs_joint_accounts,ltfs_guarantor_accounts,ltfs_curr_bal_grtr_0,ltfs_num_accounts,ltfs_mean_correctedDISBURSED-AMT/HIGH CR

In [21]:
train = df[df.ID.isin(train.ID)]
test = df[df.ID.isin(test.ID)]
train = train.sort_values('ID',ascending=True)
test = test.sort_values('ID',ascending=True)
train[target] = train[target].astype('int')

In [22]:
train.head()

,ID,Frequency,InstlmentMode,LoanStatus,PaymentMode,BranchID,Area,Tenure,AssetCost,AmountFinance,DisbursalAmount,EMI,DisbursalDate,MaturityDAte,AuthDate,AssetID,ManufacturerID,SupplierID,LTV,SEX,AGE,MonthlyIncome,City,State,ZiPCODE,Top-up Month,individual_accounts,joint_accounts,guarantor_accounts,curr_bal_grtr_0,num_accounts,mean_correctedDISBURSED-AMT/HIGH CREDIT,min_correctedDISBURSED-AMT/HIGH CREDIT,max_correctedDISBURSED-AMT/HIGH CREDIT,std_correctedDISBURSED-AMT/HIGH CREDIT,sum_correctedDISBURSED-AMT/HIGH CREDIT,mean_correctedCURRENT-BAL,min_correctedCURRENT-BAL,max_correctedCURRENT-BAL,std_correctedCURRENT-BAL,sum_correctedCURRENT-BAL,mean_correctedOVERDUE-AMT,min_correctedOVERDUE-AMT,max_correctedOVERDUE-AMT,std_correctedOVERDUE-AMT,sum_correctedOVERDUE-AMT,num_closed_accounts,num_open_accounts,num_delinq_accounts,total_written_off_amount,mean_percent_paid_off,min_percent_paid_off,max_percent_paid_off,std_percent_paid_off,overall_percent_paid_off,median_tenor,max_tenor,min_tenor,std_count,ddd_count,xxx_count,late_count,_30_count,_60_count,_90_count,_180_count,total_count,std_count_application_loan,ddd_count_application_loan,xxx_count_application_loan,late_count_application_loan,_30_count_application_loan,_60_count_application_loan,_90_count_application_loan,_180_count_application_loan,total_count_application_loan,mean_dpd_str,min_dpd_str,max_dpd_str,std_dpd_str,mean_dpd_str_application_loan,min_dpd_str_application_loan,max_dpd_str_application_loan,std_dpd_str_application_loan,num_accounts__between_0_and_365_days,total_sanctioned_amount__between_0_and_365_days,total_curr_bal__between_0_and_365_days,overall_percentage_paid_off__between_0_and_365_days,num_accounts__between_365_and_730_days,total_sanctioned_amount__between_365_and_730_days,total_curr_bal__between_365_and_730_days,overall_percentage_paid_off__between_365_and_730_days,num_accounts__between_730_and_1095_days,total_sanctioned_amount__between_730_and_1095_days,total_curr_bal__between_730_and_1095_days,overall_percentage_paid_off__between_730_and_1095_days,num_accounts__between_1095_and_1460_days,total_sanctioned_amount__between_1095_and_1460_days,total_curr_bal__between_1095_and_1460_days,overall_percentage_paid_off__between_1095_and_1460_days,num_accounts__between_1460_and_3650_days,total_sanctioned_amount__between_1460_and_3650_days,total_curr_bal__between_1460_and_3650_days,overall_percentage_paid_off__between_1460_and_3650_days,num_accounts_Tractor Loan,total_sanctioned_amount_Tractor Loan,total_curr_bal_Tractor Loan,overall_percentage_paid_off_Tractor Loan,num_accounts_Gold Loan,total_sanctioned_amount_Gold Loan,total_curr_bal_Gold Loan,overall_percentage_paid_off_Gold Loan,num_accounts_Business Loan Priority Sector Agriculture,total_sanctioned_amount_Business Loan Priority Sector Agriculture,total_curr_bal_Business Loan Priority Sector Agriculture,overall_percentage_paid_off_Business Loan Priority Sector Agriculture,num_accounts_Kisan Credit Card,total_sanctioned_amount_Kisan Credit Card,total_curr_bal_Kisan Credit Card,overall_percentage_paid_off_Kisan Credit Card,num_accounts_Auto Loan (Personal),total_sanctioned_amount_Auto Loan (Personal),total_curr_bal_Auto Loan (Personal),overall_percentage_paid_off_Auto Loan (Personal),num_accounts_Personal Loan,total_sanctioned_amount_Personal Loan,total_curr_bal_Personal Loan,overall_percentage_paid_off_Personal Loan,num_accounts_Other,total_sanctioned_amount_Other,total_curr_bal_Other,overall_percentage_paid_off_Other,num_accounts_Overdraft,total_sanctioned_amount_Overdraft,total_curr_bal_Overdraft,overall_percentage_paid_off_Overdraft,mean_day_start_day_diff_app_vs_other,min_day_start_day_diff_app_vs_other,max_day_start_day_diff_app_vs_other,std_day_start_day_diff_app_vs_other,mean_days_bw_loans,min_days_bw_loans,max_days_bw_loans,std_days_bw_loans,sum_days_bw_loans,ltfs_individual_accounts,ltfs_joint_accounts,ltfs_guarantor_accounts,ltfs_curr_bal_grtr_0,ltfs_num_accounts,ltfs_mean_correctedDISBURSED-AMT/HIGH CR

In [23]:
del df,bur_df,bur_df2,bur_df3

In [24]:
import gc
gc.collect()

44

In [25]:
folds = StratifiedKFold(5, shuffle = True, random_state = 2)
folds = [(x,y) for x,y in folds.split(train,train[target])]

In [26]:
len(use_cols)

1312

In [27]:
lgbm_space = {
    "n_estimators": 5000,
    "num_leaves": hp.quniform("num_leaves", 32, 192, 32),
    "min_child_weight": hp.quniform("min_child_weight", 10, 100, 20),
    "subsample": hp.quniform("subsample", 0.5, 1, 0.1),
    "colsample_bytree": hp.quniform("colsample_bytree", 0.5, 1, 0.1),
    "subsample_freq": 5,
    "objective": "multiclass",
    "boosting_type": "gbdt",
    "learning_rate": 0.1,
    "n_jobs": -1,
}

In [28]:
est = Estimator(model = LGBMClassifier(),validation_scheme=folds,early_stopping_rounds=100,n_jobs=-1)

In [29]:
%%time
hyp= HyperOptModelSelection(model=est,space=lgbm_space,max_evals=50,is_maximize=True,log_file_path='hyp4.txt')

CPU times: user 1.69 ms, sys: 0 ns, total: 1.69 ms
Wall time: 783 µs


In [30]:
%%time
hyp.fit(train[use_cols].values,train[target].values)

Starting HyperOpt 50 Evals with Dataset of Shape ((128655, 1312),(128655,))


  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]


Iteration: 1, Training with params: {'boosting_type': 'gbdt', 'colsample_bytree': 0.7000000000000001, 'learning_rate': 0.1, 'min_child_weight': 100.0, 'n_estimators': 5000, 'n_jobs': -1, 'num_leaves': 64, 'objective': 'multiclass', 'subsample': 0.6000000000000001, 'subsample_freq': 5}


Training until validation scores don't improve for 100 rounds
[100]	valid_0's multi_logloss: 0.302891	valid_1's multi_logloss: 0.260484
[200]	valid_0's multi_logloss: 0.299164	valid_1's multi_logloss: 0.226634
[300]	valid_0's multi_logloss: 0.299205	valid_1's multi_logloss: 0.201833
[400]	valid_0's multi_logloss: 0.300337	valid_1's multi_logloss: 0.181699
Early stopping, best iteration is:                    
[305]	valid_0's multi_logloss: 0.29904	valid_1's multi_logloss: 0.200693
Training until validation scores don't improve for 100 rounds
[100]	valid_0's multi_logloss: 0.298382	valid_1's multi_logloss: 0.261686
[200]	valid_0's multi_logloss: 0.293612	valid_1's multi_logloss: 0.227196
[300]	valid_0's multi_logloss: 0.293304	valid_1's multi_logloss: 0.2022
Early stopping, best iteration is:                    
[283]	valid_0's multi_logloss: 0.293026	valid_1's multi_logloss: 0.206017
Training until validation scores don't improve for 100 rounds
[100]	valid_0's multi_logloss: 0.297027	v

Score - 0.6988788284568069, Std - 0.005719308055845754, Eval Score - 0.6988788284568069
Score across folds - [0.6944770026982514, 0.6943081716066027, 0.6980801250414668, 0.6976443935345726, 0.7098844494031417].


...

Training until validation scores don't improve for 100 rounds                        
[100]	valid_0's multi_logloss: 0.287349	valid_1's multi_logloss: 0.143203            
Early stopping, best iteration is:                                                   
[89]	valid_0's multi_logloss: 0.286918	valid_1's multi_logloss: 0.153787
Training until validation scores don't improve for 100 rounds                        
[100]	valid_0's multi_logloss: 0.281882	valid_1's multi_logloss: 0.144232            
Early stopping, best iteration is:                                                   
[77]	valid_0's multi_logloss: 0.281284	valid_1's multi_logloss: 0.168163
Training until validation scores don't improve for 100 rounds                        
[100]	valid_0's multi_logloss: 0.281925	valid_1's multi_logloss: 0.143247            
Early stopping, best iteration is:                                                   
[88]	valid_0's multi_logloss: 0.281415	valid_1's multi_logloss: 0.154947
Trainin

Score - 0.715558450282308, Std - 0.004267070899768524, Eval Score - 0.715558450282308
Score across folds - [0.7092802203888325, 0.7146855169521766, 0.7169189193920037, 0.7144385550199763, 0.7224690396585511].


100%|██████████| 50/50 [9:46:47<00:00, 704.14s/trial, best loss: 0.27683595757149904]

Best Score- 0.723164042428501, Best Params- {'boosting_type': 'gbdt', 'colsample_bytree': 0.8, 'learning_rate': 0.1, 'min_child_weight': 20.0, 'n_estimators': 5000, 'n_jobs': -1, 'num_leaves': 64, 'objective': 'multiclass', 'subsample': 1.0, 'subsample_freq': 5}



CPU times: user 4d 18h 25min 46s, sys: 20min 13s, total: 4d 18h 45min 59s
Wall time: 9h 46min 47s


In [31]:
# params={'boosting_type': 'gbdt', 'colsample_bytree': 0.8, 'learning_rate': 0.1, 'min_child_weight': 40.0, 'n_estimators': 5000,
#         'n_jobs': -1, 'num_leaves': 96, 'objective': 'multiclass', 'subsample': 1.0, 'subsample_freq': 5}

params = {'boosting_type': 'gbdt', 'colsample_bytree': 0.8, 'learning_rate': 0.1, 'min_child_weight': 20.0, 'n_estimators': 5000,
 'n_jobs': -1, 'num_leaves': 64, 'objective': 'multiclass', 'subsample': 1.0, 'subsample_freq': 5}

In [32]:
# est = Estimator(model = LGBMClassifier(colsample_bytree=0.7000000000000001, min_child_weight=60.0,learning_rate=0.1,
#                n_estimators=5000, num_leaves=64, objective='multiclass',metric='None',
#                subsample_freq=5),validation_scheme=folds,early_stopping_rounds=100,n_jobs=-1)

est = Estimator(model = LGBMClassifier(**params),validation_scheme=folds,early_stopping_rounds=100,n_jobs=-1)

In [33]:
temp = est.fit_transform(train[use_cols].values,train[target].values)

Training until validation scores don't improve for 100 rounds
[100]	valid_0's multi_logloss: 0.28142	valid_1's multi_logloss: 0.163046
[200]	valid_0's multi_logloss: 0.286845	valid_1's multi_logloss: 0.103499
Early stopping, best iteration is:
[100]	valid_0's multi_logloss: 0.28142	valid_1's multi_logloss: 0.163046
Training until validation scores don't improve for 100 rounds
[100]	valid_0's multi_logloss: 0.275744	valid_1's multi_logloss: 0.164677
[200]	valid_0's multi_logloss: 0.280466	valid_1's multi_logloss: 0.104681
Early stopping, best iteration is:
[104]	valid_0's multi_logloss: 0.275572	valid_1's multi_logloss: 0.161335
Training until validation scores don't improve for 100 rounds
[100]	valid_0's multi_logloss: 0.276966	valid_1's multi_logloss: 0.163289
Early stopping, best iteration is:
[99]	valid_0's multi_logloss: 0.276931	valid_1's multi_logloss: 0.164091
Training until validation scores don't improve for 100 rounds
[100]	valid_0's multi_logloss: 0.278501	valid_1's multi_lo

In [34]:
est.cv_scores,est.overall_cv_score

([0.7181989038258415,
  0.7234888597691695,
  0.7218229007719303,
  0.7214905519100375,
  0.7308189958655257],
 0.7232557490284209)

In [35]:
train[target].value_counts(1)

0    0.829171
6    0.065027
5    0.028417
3    0.027142
4    0.023800
2    0.018406
1    0.008037
Name: Top-up Month, dtype: float64

In [36]:
feat_imp = est.feature_importance_df(use_cols)

In [37]:
feat_imp.head(50)

,column,feature_importance,rank
21,MonthlyIncome,0.021489,1
1301,cal_tenor,0.017151,2
12,State,0.015426,3
1302,emi_sal_ratio,0.015421,4
20,AGE,0.015071,5
11,City,0.014839,6
6,SupplierID,0.014162,7
13,BranchID,0.013588,8
9,pin3,0.013224,9
7,pin1,0.012773,10


In [ ]:
# for i in [100,300,500,700,900]:
#     col = feat_imp.iloc[:i]['column']
#     temp = est.fit_transform(train[col].values,train[target].values)
#     print(est.overall_cv_score)

In [38]:
pd.Series(temp.argmax(axis=1)).value_counts(1)

0    0.887948
6    0.044996
5    0.018079
3    0.015911
4    0.014038
2    0.012631
1    0.006397
dtype: float64

In [39]:
f1_score(train[target],pd.Series(temp.argmax(axis=1)),average='macro')

0.7232557490284209

In [40]:
test_preds = [est.predict_proba(test[use_cols].values) for est in est.fitted_models]

In [41]:
test['Top-up Month'] = np.sum(test_preds,axis=0).argmax(axis=1)

In [42]:
test['Top-up Month'].value_counts(1)

0    0.920515
6    0.035402
5    0.017294
4    0.008138
2    0.006714
3    0.006443
1    0.005493
Name: Top-up Month, dtype: float64

In [43]:
# test['Top-up Month'].value_counts(1)

In [ ]:
# test['Top-up Month'].value_counts(1)

In [44]:
sub = test[['ID','Top-up Month']]

In [45]:
sub['Top-up Month'] = sub['Top-up Month'].map(reverse_map)

In [46]:
sub.to_csv('LGBM_CV0.7232.csv',index=False)